In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder



In [3]:
base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")

In [9]:
#base_treino = base_treino[base_treino['proprietario']=='zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz']

#x_teste_df = x_teste_df[x_teste_df['proprietario'] == 'zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz']

In [6]:


#x_teste_df = x_teste_df[x_teste_df['proprietario'] == 'zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz']

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#nm_prop_oportunidade,dias_desejo,dias_em_casa	

In [5]:
base_treino

,id_quote,dt_criacao,dias_desejo,dias_em_casa,ds_tipo_obra,nm_prop_oportunidade,proprietario,promocional,convertido,tipo_loja,...,amb_outros,revestimento,officina,loucas_metais,arg_rejunte,categ_outros,vl_frete,vlr_orcamento,convertidos,nao_convertidos
0,0Q03s000000GRoxCAG,2022-06-09,203,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,N,Tradicional A,...,S,7,2,0,8,0,153.33,76444.94,412,330
1,0Q03s000000GRsGCAW,2022-06-09,11,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,S,Tradicional A,...,N,1,0,0,0,0,26.67,64.80,412,330
2,0Q03s000000GRwXCAW,2022-06-09,15,534,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,N,Tradicional A,...,S,1,0,0,0,0,266.67,14763.62,343,438
3,0Q03s000000GS0UCAW,2022-06-09,30,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,S,Tradicional A,...,S,1,0,0,4,0,66.67,1301.09,412,330
4,0Q03s000000GS1cCAG,2022-06-09,29,534,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,N,Tradicional A,...,S,0,0,10,0,0,83.33,3276.30,342,406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,0Q03s000001mURcCAM,2023-10-17,24,39,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,N,Tradicional A,...,S,1,0,0,4,0,93.33,1905.18,343,438
5593,0Q03s000001mUX1CAM,2023-10-17,1,39,Reforma,patricia agnes,zzzzzzzzXzzzzz,S,S,Tradicional A,...,S,1,0,0,0,0,60.00,131.27,348,503
5594,0Q03s000001mUdiCAE,2023-10-17,52,39,Nova Obra,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,N,Tradicional A,...,S,8,0,0,10,0,493.33,35609.59,488,538
5595,0Q03s000001mUh6CAE,2023-10-17,196,39,Reforma,matheus homrich,zzzzzzzXzzzzzzz,S,S,Tradicional A,...,S,7,4,0,18,0,576.45,27856.07,184,197


In [ ]:
"""TESTE REV 1"""

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")

y_teste = x_teste_df['convertido']
x_teste_df = x_teste_df.drop(['convertido', 'id_quote'], axis=1)

categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']

date_column = 'dt_criacao'

if date_column in base_treino.columns:
    print(f"Removing date column: {date_column}")
    base_treino = base_treino.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the training data.")

if date_column in x_teste_df.columns:
    x_teste_df = x_teste_df.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the test data.")

X = base_treino.drop(['convertido'], axis=1, errors='ignore')
y = base_treino['convertido']


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categoricos]).toarray(),
                          columns=encoder.get_feature_names_out(categoricos))

X_final = pd.concat([X.drop(categoricos, axis=1), X_encoded], axis=1)

X_test_encoded = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                               columns=encoder.get_feature_names_out(categoricos))
X_test_final = pd.concat([x_teste_df.drop('id_quote', axis=1, errors='ignore'), X_test_encoded], axis=1).drop(categoricos, axis=1).fillna(0)

X = X.select_dtypes(exclude=['datetime64[ns]']) 

numeric_columns = X_final.select_dtypes(include=[np.number]).columns
var_threshold = 0.05
selector = VarianceThreshold(threshold=var_threshold)
X_high_variance = selector.fit_transform(X_final[numeric_columns])

selected_feature_indices = selector.get_support(indices=True)
X_resampled = X_high_variance
X_test_final = X_test_final.iloc[:, selected_feature_indices]

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
}

xgb_classifier = XGBClassifier()

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(xgb_classifier, param_grid, scoring='accuracy', cv=cv, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)

best_params = grid_search.best_params_

modelo_xgb = XGBClassifier(**best_params)

modelo_xgb.fit(X_resampled, y_resampled)

probabilidades_teste = modelo_xgb.predict_proba(X_test_final)[:, 1]



Removing date column: dt_criacao


In [4]:
probabilidades_teste = modelo_xgb.predict_proba(X_test_final)[:, 1]

In [16]:
df_temporario_teste = pd.DataFrame({
    'nm_prop_oportunidade': x_teste_df.loc[x_teste_df.index, 'nm_prop_oportunidade'],
    'convertido': y_teste,
    'probabilidade_conversao': probabilidades_teste
   
   
})


df_temporario_teste = df_temporario_teste.drop_duplicates(subset=['nm_prop_oportunidade', 'convertido'])
df_resultados_teste = df_temporario_teste.groupby('nm_prop_oportunidade', as_index=False).agg({
    'convertido': 'first',
    'probabilidade_conversao': 'mean'
})


with pd.option_context('display.max_rows', None):
    display(df_resultados_teste)

,nm_prop_oportunidade,convertido,probabilidade_conversao
0,adryson pinto freitas,S,0.495354
1,ediane rosa,S,0.946118
2,fabiano martins werutsky,S,0.546198
3,felipe pedroso alves,S,0.925496
4,juliana westphalen gonçalves,N,0.388855
5,matheus homrich,S,0.822305
6,michelle costa malta,N,0.819763
7,rafaela de sousa paonessa loureiro,S,0.765156


In [ ]:
df_resultados_teste.to_csv("testeprevisao.csv")

In [8]:
"""TESTE REV 2"""

'TESTE REV 2'

In [9]:
base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_score, recall_score, f1_score


base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")


date_column = 'dt_criacao'
for df in [base_treino, x_teste_df]:
    if date_column in df.columns:
        print(f"Removendo coluna de data: {date_column}")
        df.drop(date_column, axis=1, errors='ignore', inplace=True)


categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']


y = base_treino['convertido']


label_encoder_y = LabelEncoder()
y = label_encoder_y.fit_transform(y)


X = base_treino.drop([date_column], axis=1, errors='ignore')


encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categoricos]).toarray(),
                          columns=encoder.get_feature_names_out(categoricos))
X_final = pd.concat([X.drop(categoricos, axis=1), X_encoded], axis=1)


numeric_columns = X_final.select_dtypes(include=[np.number]).columns
var_threshold = 0.20
selector = VarianceThreshold(threshold=var_threshold)
X_high_variance = selector.fit_transform(X_final[numeric_columns])

selected_feature_indices = selector.get_support(indices=True)
X_high_variance_columns = X_final.columns[selected_feature_indices]


X_high_variance = pd.DataFrame(X_high_variance, columns=X_high_variance_columns)


smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_high_variance, y)




param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
}


xgb_classifier = XGBClassifier()


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


grid_search = GridSearchCV(xgb_classifier, param_grid, scoring='accuracy', cv=cv, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)


best_params = grid_search.best_params_


modelo_xgb = XGBClassifier(**best_params)


modelo_xgb.fit(X_resampled, y_resampled)


calibrated_model = CalibratedClassifierCV(modelo_xgb, method='sigmoid', cv='prefit')
calibrated_model.fit(X_resampled, y_resampled)


y_pred_train = calibrated_model.predict(X_resampled)
print("Métricas no Conjunto de Treinamento:")
print("Precisão: ", precision_score(y_resampled, y_pred_train))
print("Revocação: ", recall_score(y_resampled, y_pred_train))
print("Pontuação F1: ", f1_score(y_resampled, y_pred_train))


X_test_encoded = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                              columns=encoder.get_feature_names_out(categoricos))
X_test_final = pd.concat([x_teste_df.drop(categoricos, axis=1), X_test_encoded], axis=1)


numeric_columns_test = X_test_final.select_dtypes(include=[np.number]).columns
X_test_high_variance = selector.transform(X_test_final[numeric_columns_test])


X_test_high_variance = pd.DataFrame(X_test_high_variance, columns=X_high_variance_columns)


probabilidades_teste = calibrated_model.predict_proba(X_test_high_variance)[:, 1]



Removendo coluna de data: dt_criacao
Removendo coluna de data: dt_criacao
Métricas no Conjunto de Treinamento:
Precisão:  0.7739380849532037
Revocação:  0.7514854945823138
Pontuação F1:  0.7625465508068806


In [7]:
df_temporario_teste = pd.DataFrame({
    'nm_prop_oportunidade': x_teste_df.loc[x_teste_df.index, 'nm_prop_oportunidade'],
    'convertido': y_teste,
    'probabilidade_conversao': probabilidades_teste,
    'id_quote':x_teste_df.loc[x_teste_df.index, 'id_quote']
})


df_temporario_teste = df_temporario_teste.drop_duplicates(subset=['nm_prop_oportunidade', 'convertido','id_quote'])
df_resultados_teste = df_temporario_teste.groupby('nm_prop_oportunidade', as_index=False).agg({
    'convertido': 'first',
    'id_quote':'first',
    'probabilidade_conversao': 'mean'
})


with pd.option_context('display.max_rows', None):
    display(df_resultados_teste)

,nm_prop_oportunidade,convertido,id_quote,probabilidade_conversao
0,adryson pinto freitas,S,0Q03s000000T3ACCA0,0.308874
1,ediane rosa,S,0Q0SG0000001dsB0AQ,0.913264
2,fabiano martins werutsky,S,0Q03s000000T1SnCAK,0.476274
3,felipe pedroso alves,S,0Q0SG0000003JR80AM,0.865123
4,juliana westphalen gonçalves,N,0Q03s000000T1zXCAS,0.452615
5,matheus homrich,S,0Q03s000000T2EDCA0,0.643881
6,michelle costa malta,N,0Q0SG0000003voA0AQ,0.905814
7,rafaela de sousa paonessa loureiro,S,0Q03s000000T1aNCAS,0.613912


In [14]:
df_resultados_teste.to_csv("testeprevisao.csv")

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV


base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")


y_teste = x_teste_df['convertido']
x_teste_df = x_teste_df.drop('convertido', axis=1)


categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']
date_column = 'dt_criacao'


date_column_stripped = date_column.strip()
if date_column_stripped in categoricos:
    print(f"Removing date column: {date_column_stripped}")
    base_treino = base_treino.drop(date_column_stripped, axis=1)
    x_teste_df = x_teste_df.drop(date_column_stripped, axis=1)
    categoricos.remove(date_column_stripped)
else:
    print(f"Date column '{date_column_stripped}' not found in the categorical columns list.")


if date_column in base_treino.columns:
    base_treino[date_column] = pd.to_datetime(base_treino[date_column])
    x_teste_df[date_column] = pd.to_datetime(x_teste_df[date_column])


X_no_date = base_treino.drop(['convertido', 'id_quote', date_column], axis=1)
X_date_only = pd.DataFrame(base_treino[date_column])


encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
df_encoded_train = pd.DataFrame(encoder.fit_transform(X_no_date[categoricos]).toarray(),
                                columns=encoder.get_feature_names_out(categoricos))
df_final_train = pd.concat([X_date_only, df_encoded_train], axis=1)


df_encoded_test = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                               columns=encoder.get_feature_names_out(categoricos))
x_teste_df_encoded = pd.concat([x_teste_df[date_column].astype('int64'), df_encoded_test], axis=1)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(base_treino['convertido'])

var_threshold = 0.05
selector = VarianceThreshold(threshold=var_threshold)
X_high_variance = selector.fit_transform(df_final_train.drop([date_column], axis=1))

if X_high_variance.shape[1] == 0:
    raise ValueError(f"No feature in X meets the variance threshold {var_threshold:.5f}")

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_high_variance, y)


X_resampled_columns = X_high_variance.columns  
X_resampled = pd.DataFrame(X_resampled, columns=X_resampled_columns)


common_features = set(X_resampled.columns) & set(x_teste_df_encoded.columns)
X_resampled = X_resampled[common_features]
x_teste_df_encoded = x_teste_df_encoded[common_features]


param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
}

grad_boost = GradientBoostingClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(grad_boost, param_grid, scoring='accuracy', cv=cv, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)


best_params = grid_search.best_params_
modelo_grad_boost = GradientBoostingClassifier(**best_params)
modelo_grad_boost.fit(X_resampled, y_resampled)


probabilidades_teste = modelo_grad_boost.predict_proba(x_teste_df_encoded)[:, 1]


df_temporario_teste = pd.DataFrame({
    'nm_prop_oportunidade': x_teste_df.loc[x_teste_df.index, 'nm_prop_oportunidade'],
    'convertido': y_teste,
    'probabilidade_conversao': probabilidades_teste
})


df_temporario_teste = df_temporario_teste.drop_duplicates(subset=['nm_prop_oportunidade', 'convertido'])
df_resultados_teste = df_temporario_teste.groupby('nm_prop_oportunidade', as_index=False).agg({
    'convertido': 'first',
    'probabilidade_conversao': 'mean'
})


with pd.option_context('display.max_rows', None):
    display(df_resultados_teste)


Date column 'dt_criacao' not found in the categorical columns list.


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
print("Columns after dropping date column:")
print(X_no_date.columns)
print("Columns after one-hot encoding:")
print(df_encoded_train.columns)



Columns after dropping date column:
Index(['dias_desejo', 'dias_em_casa', 'ds_tipo_obra', 'nm_prop_oportunidade',
       'proprietario', 'promocional', 'tipo_loja', 'banheiro', 'toda_casa',
       'cozinha', 'area_externa', 'itens_assentamento', 'sala', 'piscina',
       'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada',
       'amb_outros', 'revestimento', 'officina', 'loucas_metais',
       'arg_rejunte', 'categ_outros', 'vl_frete', 'vlr_orcamento',
       'convertidos', 'nao_convertidos'],
      dtype='object')
Columns after one-hot encoding:
Index(['ds_tipo_obra_Nova Obra', 'ds_tipo_obra_Reforma', 'ds_tipo_obra_nan',
       'nm_prop_oportunidade_adryson pinto freitas',
       'nm_prop_oportunidade_ana paula goncalves lemos',
       'nm_prop_oportunidade_bruno schmelzer de souza',
       'nm_prop_oportunidade_claudia marina silva',
       'nm_prop_oportunidade_dara jenifer de jesus',
       'nm_prop_oportunidade_ediane rosa',
       'nm_prop_oportunidade_fabiano martins w